In [1]:
import numpy as np
import pandas as pd

In [2]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout

In [3]:
(x_train,y_train),(x_test,y_test) = keras.datasets.mnist.load_data()

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


In [4]:
x_train.shape

(60000, 28, 28)

In [5]:
x_train=x_train/255
x_test=x_test/255

In [6]:
model = Sequential()

model.add(Flatten(input_shape=(28,28)))
model.add(Dense(256,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dense(10,activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ flatten (Flatten)                    │ (None, 784)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         200,960 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 128)                 │          32,896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │           1,290 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 235,146 (918.54 KB)

 Trainable params: 235,146 (918.54 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

In [9]:
model.fit(x_train,y_train,epochs=10,validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.8548 - loss: 0.4712 - val_accuracy: 0.9622 - val_loss: 0.1245
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9559 - loss: 0.1419 - val_accuracy: 0.9703 - val_loss: 0.1044
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.9677 - loss: 0.1036 - val_accuracy: 0.9722 - val_loss: 0.0930
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9739 - loss: 0.0844 - val_accuracy: 0.9692 - val_loss: 0.1014
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9762 - loss: 0.0740 - val_accuracy: 0.9750 - val_loss: 0.0831
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9799 - loss: 0.0630 - val_accuracy: 0.9768 - val_loss: 0.0849
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9822 - loss: 0.0561 - val_accuracy: 0.9783 - val_loss: 0.0826
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.9819 - loss: 0.0528 - 

In [10]:
y_prob = model.predict(x_test)

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step


In [11]:
y_pred = y_prob.argmax(axis = 1)

In [12]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9807

In [13]:
def create_shards(x_train: np.ndarray, y_train: np.ndarray, num_shards: int):
    num_samples = x_train.shape[0]
    shard_size = num_samples // num_shards
    shards = []

    # Distribute samples evenly to each shard
    for i in range(num_shards):
        start_idx = i * shard_size
        # Ensure the last shard gets all remaining samples
        end_idx = start_idx + shard_size if i != num_shards - 1 else num_samples

        x_shard = x_train[start_idx:end_idx]
        y_shard = y_train[start_idx:end_idx]

        shards.append((x_shard, y_shard))

    # Calculate the shape to which all shards should be adjusted
    x_shard_shape = shards[0][0].shape
    y_shard_shape = shards[0][1].shape

    # Pad or truncate shards to make all of them have the same shape
    adjusted_shards = []
    for x_shard, y_shard in shards:
        # Truncate or pad x_shard
        if x_shard.shape != x_shard_shape:
            x_shard = np.resize(x_shard, x_shard_shape)

        # Truncate or pad y_shard
        if y_shard.shape != y_shard_shape:
            y_shard = np.resize(y_shard, y_shard_shape)

        adjusted_shards.append((x_shard, y_shard))

    # Stack the adjusted shards to form numpy arrays
    x_shards = np.stack([shard[0] for shard in adjusted_shards], axis=0)
    y_shards = np.stack([shard[1] for shard in adjusted_shards], axis=0)

    return adjusted_shards, x_shards, y_shards

In [14]:
num_shards = 3
shards, x_shards, y_shards = create_shards(x_train, y_train, num_shards)

In [15]:
x_shards.shape

(3, 20000, 28, 28)

In [16]:
y_shards.shape

(3, 20000)

In [17]:
x_shards[0][0]

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.    

In [18]:
def create_model():
    model = Sequential()
    model.add(Flatten(input_shape=(28,28)))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(128,activation='relu'))
    model.add(Dense(10,activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',optimizer='Adam', metrics=['accuracy'])

    return model

In [19]:
aggregation = []

for i in range(num_shards):
    model = create_model()

    model.fit(x_shards[i], y_shards[i],epochs=10,validation_split=0.2)

    aggregation.append(model)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


500/500 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7703 - loss: 0.7270 - val_accuracy: 0.9220 - val_loss: 0.2614
Epoch 2/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9434 - loss: 0.1992 - val_accuracy: 0.9550 - val_loss: 0.1510
Epoch 3/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9574 - loss: 0.1398 - val_accuracy: 0.9560 - val_loss: 0.1391
Epoch 4/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9666 - loss: 0.1037 - val_accuracy: 0.9638 - val_loss: 0.1289
Epoch 5/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9747 - loss: 0.0818 - val_accuracy: 0.9638 - val_loss: 0.1181
Epoch 6/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9763 - loss: 0.0688 - val_accuracy: 0.9643 - val_loss: 0.1302
Epoch 7/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9805 - loss: 0.0587 - val_accuracy: 0.9615 - val_loss: 0.1303
Epoch 8/10
500/500 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9795 - loss: 0.0619 - val_accuracy: 0.9697 - val_

In [20]:
aggregation

[<Sequential name=sequential_1, built=True>,
 <Sequential name=sequential_2, built=True>,
 <Sequential name=sequential_3, built=True>]

In [21]:
predictions = np.array([ensumable.predict(x_test) for ensumable in aggregation])
argmax_indices = np.argmax(predictions, axis=2)
model_pred = np.swapaxes(argmax_indices,0, 1)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


In [22]:
from scipy import stats

In [23]:
modes = stats.mode(model_pred, axis=1)

most_frequent_val = modes.mode.flatten()

In [24]:
most_frequent_val

array([7, 2, 1, ..., 4, 5, 6])

In [25]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,most_frequent_val)

0.9747

In [27]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.2/233.2 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.5 MB/s eta 0:00:00


In [28]:
import optuna

In [30]:
def objective(trial):
    num_shards = trial.suggest_int("num_shards",2,20)

    shards, x_shards, y_shards = create_shards(x_train, y_train, num_shards)
    aggregation_tunned = []

    for i in range(num_shards):
        model = create_model()
        model.fit(x_shards[i], y_shards[i],epochs=10,validation_split=0.2)
        aggregation_tunned.append(model)

    predictions = np.array([ensumable.predict(x_test) for ensumable in aggregation_tunned])
    argmax_indices = np.argmax(predictions, axis=2)
    model_pred = np.swapaxes(argmax_indices,0, 1)

    modes = stats.mode(model_pred, axis=1)
    most_frequent_values = modes.mode.flatten()

    accuracy = accuracy_score(y_test,most_frequent_values)

    return accuracy

In [31]:
study = optuna.create_study(direction = "maximize")

[I 2024-10-02 14:27:43,680] A new study created in memory with name: no-name-2c2436ff-8697-4a7f-a12f-b2f59b50d836


In [32]:
study.optimize(objective, n_trials = 10)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


116/116 ━━━━━━━━━━━━━━━━━━━━ 6s 38ms/step - accuracy: 0.6272 - loss: 1.2389 - val_accuracy: 0.8938 - val_loss: 0.3644
Epoch 2/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8931 - loss: 0.3496 - val_accuracy: 0.9122 - val_loss: 0.3069
Epoch 3/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9261 - loss: 0.2505 - val_accuracy: 0.9274 - val_loss: 0.2713
Epoch 4/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9500 - loss: 0.1702 - val_accuracy: 0.9393 - val_loss: 0.2195
Epoch 5/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9655 - loss: 0.1224 - val_accuracy: 0.9361 - val_loss: 0.2270
Epoch 6/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9702 - loss: 0.1133 - val_accuracy: 0.9350 - val_loss: 0.2371
Epoch 7/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9753 - loss: 0.0827 - val_accuracy: 0.9426 - val_loss: 0.2364
Epoch 8/10
116/116 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9859 - loss: 0.0523 - val_accuracy: 0.9350 - val

[I 2024-10-02 14:29:34,252] Trial 0 finished with value: 0.9581 and parameters: {'num_shards': 13}. Best is trial 0 with value: 0.9581.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 7s 28ms/step - accuracy: 0.6639 - loss: 1.0756 - val_accuracy: 0.9250 - val_loss: 0.2732
Epoch 2/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9187 - loss: 0.2924 - val_accuracy: 0.9340 - val_loss: 0.2133
Epoch 3/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9434 - loss: 0.2036 - val_accuracy: 0.9528 - val_loss: 0.1703
Epoch 4/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9566 - loss: 0.1434 - val_accuracy: 0.9505 - val_loss: 0.1639
Epoch 5/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9668 - loss: 0.1050 - val_accuracy: 0.9303 - val_loss: 0.2006
Epoch 6/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9741 - loss: 0.0895 - val_accuracy: 0.9468 - val_loss: 0.1663
Epoch 7/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9764 - loss: 0.0731 - val_accuracy: 0.9430 - val_loss: 0.1912
Epoch 8/10
167/167 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9782 - loss: 0.0629 - val_accuracy: 0

...

[I 2024-10-02 14:41:15,310] Trial 7 finished with value: 0.958 and parameters: {'num_shards': 13}. Best is trial 1 with value: 0.9623.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 6s 59ms/step - accuracy: 0.5305 - loss: 1.4761 - val_accuracy: 0.8671 - val_loss: 0.4325
Epoch 2/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8813 - loss: 0.3974 - val_accuracy: 0.8972 - val_loss: 0.3309
Epoch 3/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9239 - loss: 0.2611 - val_accuracy: 0.8972 - val_loss: 0.3341
Epoch 4/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9404 - loss: 0.2106 - val_accuracy: 0.9161 - val_loss: 0.2887
Epoch 5/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9581 - loss: 0.1634 - val_accuracy: 0.9098 - val_loss: 0.2790
Epoch 6/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9655 - loss: 0.1155 - val_accuracy: 0.9225 - val_loss: 0.2569
Epoch 7/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9753 - loss: 0.0967 - val_accuracy: 0.9209 - val_loss: 0.2643
Epoch 8/10
79/79 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9831 - loss: 0.0680 - val_accuracy: 0.9335 - val_loss

[I 2024-10-02 14:43:31,874] Trial 8 finished with value: 0.9504 and parameters: {'num_shards': 19}. Best is trial 1 with value: 0.9623.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8125 - loss: 0.6195 - val_accuracy: 0.9450 - val_loss: 0.1751
Epoch 2/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9486 - loss: 0.1732 - val_accuracy: 0.9570 - val_loss: 0.1399
Epoch 3/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9608 - loss: 0.1236 - val_accuracy: 0.9587 - val_loss: 0.1281
Epoch 4/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9700 - loss: 0.0966 - val_accuracy: 0.9627 - val_loss: 0.1201
Epoch 5/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9747 - loss: 0.0760 - val_accuracy: 0.9640 - val_loss: 0.1232
Epoch 6/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9786 - loss: 0.0652 - val_accuracy: 0.9673 - val_loss: 0.1197
Epoch 7/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9805 - loss: 0.0607 - val_accuracy: 0.9732 - val_loss: 0.1013
Epoch 8/10
750/750 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9835 - loss: 0.0515 - val_accuracy: 0.

[I 2024-10-02 14:44:06,120] Trial 9 finished with value: 0.9712 and parameters: {'num_shards': 2}. Best is trial 9 with value: 0.9712.


In [33]:
study.best_params

{'num_shards': 2}

In [38]:
study.best_value

0.9712

In [36]:
optuna.visualization.plot_slice(study)

In [37]:
optuna.visualization.plot_optimization_history(study)